In [1]:
#Imports for artificial neural network
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree


np.random.seed(42)  #Seed is necessary to ensure consistent replication of results

df = pd.read_csv('data.csv')

#Since we are resampling the data, we need a clean version for testing
df_test = pd.read_csv('data.csv')
print('df1: ', df)

df1:        Bankrupt?   ROA(C) before interest and depreciation before interest  \
0             1                                           0.370594          
1             1                                           0.464291          
2             1                                           0.426071          
3             1                                           0.399844          
4             1                                           0.465022          
...         ...                                                ...          
6814          0                                           0.493687          
6815          0                                           0.475162          
6816          0                                           0.472725          
6817          0                                           0.506264          
6818          0                                           0.493053          

       ROA(A) before interest and % after tax  \
0                   

In [2]:
#Now build decision tree classifiers
def data_info():
    df.info()
    null_counts = df.isnull().sum()
    null_columns = null_counts[null_counts > 0]
    return df.describe().transpose()

df.isnull().sum().sum()

Y=df["Bankrupt?"]
X=df.drop(["Bankrupt?"],axis=1)

################################################
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, Y)
#We resample once more to see if this improves accuracy
X_double_resampled, y_double_resampled = smote.fit_resample(X_resampled, y_resampled)

df_resampled = pd.DataFrame(X_double_resampled, y_double_resampled)
#df_resampled['Bankrupt?'] = y_double_resampled
df_resampled = df_resampled.reset_index(drop=False)
df = df_resampled

print('df2: ', df)

Y=df["Bankrupt?"]
X=df.drop(["Bankrupt?"],axis=1)
################################################

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X,Y)

importance_features = rf.feature_importances_
important_features_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance_features})
important_features_df_desc =important_features_df.sort_values(by='Importance', ascending=False)
important_features_df_asc =important_features_df.sort_values(by='Importance', ascending=True)
important_features_df_desc.head(40)
required = important_features_df_desc[important_features_df_desc.Importance>0.0105]
required_features = df[required.Feature]  
correlation_matrix = df.corr()
target_corr = correlation_matrix["Bankrupt?"].abs().sort_values(ascending=False)
target_corr = target_corr.drop("Bankrupt?")
selected_features = target_corr[target_corr>=0.1]
selected_columns = list(selected_features.index)
df_selected = df[selected_columns]
actual_corr = correlation_matrix["Bankrupt?"][selected_features.index]
positive_corr = actual_corr[actual_corr >= 0].sort_values(ascending=False)
negative_corr = actual_corr[actual_corr < 0].sort_values(ascending=True)
sorted_correlations = pd.concat([positive_corr, negative_corr])

import seaborn as sns

selexted_X = df_selected
selected_y = df["Bankrupt?"]
#X_train,X_test,y_train,y_test = train_test_split(selexted_X,selected_y,test_size = 0.2,random_state=42)
X_train,X_test_dummy,y_train,y_test_dummy = train_test_split(selexted_X,selected_y,test_size = 0.2,random_state=42)

######################################################################################################
df_test.isnull().sum().sum()

Y_clean=df_test["Bankrupt?"]
X_clean=df_test.drop(["Bankrupt?"],axis=1)

selected_X = selexted_X.columns
print('selected columns: ', selected_X)
matching_columns = selected_X.intersection(df_test.columns)
X_test_selected = df_test[matching_columns]
print('X_test_selected: ', X_test_selected)
y_test_selected = df_test["Bankrupt?"]

X_train_dummy,X_test,y_train_dummy,y_test = train_test_split(X_test_selected,y_test_selected,test_size = 0.2,random_state=42)
#X_train,X_test,y_train,y_test = train_test_split(selexted_X,selected_y,test_size = 0.2,random_state=42)
#X_train,X_test,y_train,y_test = train_test_split(selexted_X,selected_y,test_size = 0.2,random_state=42)

######################################################################################################


from sklearn.model_selection import GridSearchCV
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [10, 15, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier(max_depth =5,min_samples_split = 20,random_state=42)
grid_search_dt = GridSearchCV(estimator=dt_classifier, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search_dt.fit(X_train,y_train)

grid_search_dt.best_estimator_

best_dt = grid_search_dt.best_estimator_

dt_train_pred = best_dt.predict(X_train)
dt_test_pred = best_dt.predict(X_test)

tree_info = best_dt.tree_
num_nodes = tree_info.node_count
num_leaves = tree_info.n_leaves
num_decision = num_nodes - num_leaves

rf_classifier = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf_classifier, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search_rf.fit(X_train,y_train)


grid_search_rf.best_estimator_

best_rf = grid_search_rf.best_estimator_

y_train_pred = best_rf.predict(X_train)
y_test_pred = best_rf.predict(X_test)

#Classifier 1
classifier1 = pd.DataFrame(y_train_pred, index=X_train.index, columns=['Random Forest'])

#classifier 1a
classifier1a = pd.DataFrame(y_test_pred, index=X_test.index, columns=['Random Forest'])

df2:         Bankrupt?   ROA(C) before interest and depreciation before interest  \
0              1                                           0.464291          
1              1                                           0.464291          
2              1                                           0.464291          
3              1                                           0.464291          
4              1                                           0.464291          
...          ...                                                ...          
13193          1                                           0.464291          
13194          1                                           0.464291          
13195          1                                           0.464291          
13196          1                                           0.464291          
13197          1                                           0.464291          

        ROA(A) before interest and % after tax  \
0      

selected columns:  Index([' Cash Flow to Sales', ' Retained Earnings to Total Assets',
       ' Operating Profit Rate', ' Current Liabilities/Liability',
       ' Current Liability to Liability', ' Equity to Liability',
       ' Net Income to Total Assets', ' Quick Assets/Total Assets',
       ' Cash/Current Liability', ' Operating Expense Rate',
       ' Fixed Assets Turnover Frequency', ' Net Worth Turnover Rate (times)',
       ' Debt ratio %', ' Total Asset Return Growth Rate Ratio',
       ' Cash flow rate',
       ' Interest Coverage Ratio (Interest expense to EBIT)',
       ' ROA(A) before interest and % after tax', ' Inventory/Working Capital',
       ' Borrowing dependency', ' Total Asset Turnover',
       ' Continuous Net Profit Growth Rate', ' Total expense/Assets',
       ' Accounts Receivable Turnover', ' Net worth/Assets',
       ' Inventory/Current Liability', ' Revenue Per Share (Yuan ¥)',
       ' Current Assets/Total Assets', ' Working capitcal Turnover Rate',
       

Fitting 4 folds for each of 50 candidates, totalling 200 fits


In [3]:
#Additional imports for linear regression model
import statsmodels.api as sm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

target = 'Bankrupt?'
predictors = [col for col in df.columns if col != target]

#forward selection
def forward_selection (df, target, predictors):
    features = []
    best_features = []
    while True: 
        features_left = list(set(predictors) - set(features))
        new_val = pd.Series(index = features_left)
        for new_col in tqdm(features_left, desc="Processing Columns"):
            model = sm.OLS(df[target], sm.add_constant(df[features + [new_col]])).fit()
            new_val[new_col] = model.pvalues[new_col]
        min_p_value = new_val.min()
        if min_p_value < 0.05:
            best_feature = new_val.idxmin()
            features.append(best_feature)
            best_features.append(best_feature)
        else:
            break
        return best_features

best_predictors = forward_selection(df, target, list(X_train.columns))

final_model = sm.OLS(y_train, sm.add_constant(X_train[best_predictors])).fit()

#find accuracy 
linRegPred = final_model.predict(sm.add_constant(X_train[best_predictors]))

#Classifier 2
classifier2 = pd.DataFrame(linRegPred, columns=['linear Regression'])

classifier2a = pd.DataFrame(final_model.predict(sm.add_constant(X_test[best_predictors])), columns=['linear Regression'])

assert classifier1.index.equals(classifier2.index), "Indices do not match!"

assert classifier1a.index.equals(classifier2a.index), "Indices do not match!"



C:\Users\Ryan\AppData\Local\Temp\ipykernel_5660\3779582531.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_val = pd.Series(index = features_left)
Processing Columns: 100%|█████████████████████████████████████████████████████████████| 91/91 [00:00<00:00, 250.69it/s]


In [4]:
X_train.columns = X_train.columns.str.strip()
X_test.columns = X_test.columns.str.strip()

feature = 'Net Income to Total Assets'

x = X_train[[feature]]
xtest = X_test[[feature]]
y = y_train
ytest = y_test

x = sm.add_constant(x)
xtest = sm.add_constant(xtest)
final_model = sm.OLS(y, x).fit()

predictions = final_model.predict(x)

#Classifier 3
classifier3 = pd.DataFrame(predictions, columns=['linear Regression 2'])

classifier3a = pd.DataFrame(final_model.predict(xtest), columns=['linear Regression 2'])

assert classifier2.index.equals(classifier3.index), "Indices do not match!"

assert classifier2a.index.equals(classifier3a.index), "Indices do not match!"

In [5]:
#Now we get the classifier for logistic regression

In [6]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

model = LogisticRegression(max_iter=1000, class_weight='balanced')
# Train the model
model.fit(X_train, y_train)

# Predict the target on the test set
y_pred = model.predict(X_test)

classifier4 = pd.DataFrame(model.predict(X_train), index=X_train.index, columns=['Logistic Regression'])

classifier4a = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['Logistic Regression'])

In [7]:
#Imports for support vector machine
from sklearn import svm
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from mpl_toolkits.mplot3d import Axes3D

scaler = StandardScaler()

df['Bankrupt?'] = pd.Categorical(df['Bankrupt?'])
df_test['Bankrupt?'] = pd.Categorical(df_test['Bankrupt?'])

dependentVariable = ['Bankrupt?']

featureGroupA = [
    'Tax rate (A)',
    'Net Value Per Share (B)',
    'Net Value Per Share (A)',
    'Net Value Per Share (C)',
    'Persistent EPS in the Last Four Seasons',
    'Operating Profit Per Share (Yuan ¥)',
    'Per Share Net profit before tax (Yuan ¥)',
    'Debt ratio %',
    'Operating profit/Paid-in capital',
    'Net profit before tax/Paid-in capital',
    'Quick Assets/Total Assets',
    'Cash/Total Assets',
    'Current Liability to Assets',
    'Total expense/Assets',
    'Equity to Long-term Liability',
    'Liability-Assets Flag',
    'Equity to Liability'
    ]

df.columns = df.columns.str.strip().str.lower()
df_test.columns = df_test.columns.str.strip().str.lower()
#X.columns = X.columns.str.strip().str.lower()
independentVariablesA = [col.lower() for col in featureGroupA]

#X['independentVariableA'] = X[independentVariablesA].mean(axis=1)
df['independentVariableA'] = df[independentVariablesA].mean(axis=1)
df_test['independentVariableA'] = df_test[independentVariablesA].mean(axis=1)

featureGroupB = [
    'Operating Gross Margin',
    'Realized Sales Gross Margin',
    'Net worth/Assets',
    'Working Capital to Total Assets',
    'Working Capital/Equity',
    'Retained Earnings to Total Assets',
    'Current Liability to Current Assets',
    'Net Income to Total Assets',
    'Gross Profit to Sales',
    'Net Income to Stockholder\'s Equity'
]

independentVariablesB = [col.lower() for col in featureGroupB]
df['independentVariableB'] = df[independentVariablesB].mean(axis=1)
#X['independentVariableB'] = X[independentVariablesB].mean(axis=1)
df_test['independentVariableB'] = df_test[independentVariablesB].mean(axis=1)

featureGroupC = [
    'CFO to Assets',
    'Borrowing dependency',
    'Operating profit per person',
    'ROA(A) before interest and % after tax',
    'ROA(B) before interest and depreciation after tax',
    'ROA(C) before interest and depreciation before interest',
    'Current Liabilities/Equity',
    'Current Liability to Equity',
    'Liability to Equity'
]

independentVariablesC = [col.lower() for col in featureGroupC]
df['independentVariableC'] = df[independentVariablesC].mean(axis=1)
#X['independentVariableC'] = X[independentVariablesC].mean(axis=1)
df_test['independentVariableC'] = df_test[independentVariablesC].mean(axis=1)

dependentVariable = [col.lower() for col in dependentVariable]

X = df[['independentVariableA', 'independentVariableB', 'independentVariableC']]
X1 = df_test[['independentVariableA', 'independentVariableB', 'independentVariableC']]

#dependentVariable = 'bankrupt?'
#Y = Y[dependentVariable]

Y = Y.values.ravel()

#In order to match instances with the currently in use X_train data, we grab the indices
trainIndices = X_train.index
#Same with the test data
testIndices = X_test.index
trainX = X.loc[trainIndices]
trainY = Y[trainIndices]
#testX = X.drop(trainIndices)
testX = X1.loc[testIndices]
testY = np.delete(Y, trainIndices)

polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", svm.LinearSVC(C=100000, loss="squared_hinge", max_iter=10000))
])

polynomial_svm_clf.fit(trainX, trainY)

polynomial_svm_clf2 = Pipeline([
    ("poly_features", PolynomialFeatures(degree=15)),
    ("scaler", StandardScaler()),
    ("svm_clf", svm.LinearSVC(C=1000, loss="hinge", max_iter=1000))
])

polynomial_svm_clf2.fit(trainX, trainY)

print('Evaluation: ')
trainPredictions = polynomial_svm_clf.predict(trainX)
#Classifier 5
classifier5 = pd.DataFrame(trainPredictions, index=X_train.index, columns=['Support Vector Machine 1'])


testPredictions = polynomial_svm_clf.predict(testX)
classifier5a = pd.DataFrame(testPredictions, index=X_test.index, columns=['Support Vector Machine 1'])

trainPredictions2 = polynomial_svm_clf2.predict(trainX)
#Classifier 6
classifier6 = pd.DataFrame(trainPredictions2, index=X_train.index, columns=['Support Vector Machine 2'])

testPredictions2 = polynomial_svm_clf2.predict(testX)
classifier6a = pd.DataFrame(testPredictions2, index=X_test.index, columns=['Support Vector Machine 2'])


Evaluation: 


In [8]:
print('classifier1: ', classifier1)
print('classifier2: ', classifier2)
print('classifier3: ', classifier3)
print('classifier4: ', classifier4)
print('classifier5: ', classifier5)
print('classifier6: ', classifier6)

assert classifier3.index.equals(classifier2.index), "Indices do not match!"
assert classifier4.index.equals(classifier3.index), "Indices do not match!"
assert classifier5.index.equals(classifier4.index), "Indices do not match!"
assert classifier6.index.equals(classifier5.index), "Indices do not match!"

print('classifier1a: ', classifier1a)
print('classifier2a: ', classifier2a)
print('classifier3a: ', classifier3a)
print('classifier4a: ', classifier4a)
print('classifier5a: ', classifier5a)
print('classifier6a: ', classifier6a)

assert classifier3a.index.equals(classifier2a.index), "Indices do not match!"
assert classifier4a.index.equals(classifier3a.index), "Indices do not match!"
assert classifier5a.index.equals(classifier4a.index), "Indices do not match!"
assert classifier6a.index.equals(classifier5a.index), "Indices do not match!"

train_df = pd.concat([classifier1, classifier2, classifier3, classifier4, classifier5, classifier6], axis=1)
test_df = pd.concat([classifier1a, classifier2a, classifier3a, classifier4a, classifier5a, classifier6a], axis=1)

print(train_df.head())
print(test_df.head())

classifier1:         Random Forest
8928               1
6847               1
4292               0
1042               0
2389               0
...              ...
11964              1
5191               0
5390               0
860                0
7270               1

[10558 rows x 1 columns]
classifier2:         linear Regression
8928        1.000000e+00
6847        1.000000e+00
4292        7.349676e-14
1042        7.349676e-14
2389        7.349676e-14
...                  ...
11964       1.000000e+00
5191        7.349676e-14
5390        7.349676e-14
860         7.349676e-14
7270        1.000000e+00

[10558 rows x 1 columns]
classifier3:         linear Regression 2
8928          1.000000e+00
6847          1.000000e+00
4292         -2.060574e-13
1042         -2.060574e-13
2389         -2.060574e-13
...                    ...
11964         1.000000e+00
5191         -2.060574e-13
5390         -2.060574e-13
860          -2.060574e-13
7270          1.000000e+00

[10558 rows x 1 columns]
clas

In [9]:
#Now run the classifiers through a neural network and evaluate their performance

#The model is too sensitive to non-bankruptcies, which requires resampling to allow it to train for bankrupcty instances
#Since the model is biased towards non-bankruptcies, we need to oversample the bankrupt class and undersample non-bankrupt

#from imblearn.over_sampling import SMOTE

#smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(train_df, y_train)
#We resample once more to see if this improves accuracy
X_doublesampled, y_doublesampled = smote.fit_resample(X_resampled, y_resampled)

#First classifier will use a more moderate approach
clf = MLPClassifier(solver='sgd', alpha=.1, activation='logistic', momentum=.9,
                    hidden_layer_sizes=(10, 10), random_state=1, max_iter=8000, learning_rate='constant', learning_rate_init=.2)

#Second classifier has 35 neurons in each hidden layer and a learning rate of .2
hefty_clf = MLPClassifier(solver='sgd', alpha=.2, activation='logistic',
                    hidden_layer_sizes=(35, 35), random_state=1, max_iter=5000, learning_rate='adaptive', learning_rate_init=.02)

fitted_model = clf.fit(train_df, y_train)
#Fit the models
resampled_model = clf.fit(X_resampled, y_resampled)
#hefty_fitted_model = hefty_clf.fit(train_df, y_train)
doubleresampled_model = hefty_clf.fit(X_doublesampled, y_doublesampled)

predictY = fitted_model.predict(test_df)
resampled_predictY = resampled_model.predict(test_df)
doubleresampled_predictY = doubleresampled_model.predict(test_df)

In [10]:
#The accuracy scores only assess the percentage of each of the two split variables was correct
#Since survived_0 and survived_1 are reciprocals of each other, this doesn't tell us much
#The scores are the same for each
bankrupt = accuracy_score(y_test, predictY)
bankrupt *= 100

resampled_bankrupt = accuracy_score(y_test, resampled_predictY)

doubleresampled_bankrupt = accuracy_score(y_test, doubleresampled_predictY)
doubleresampled_bankrupt *= 100

#We build a confusion matrix in order to assess the percentage of actual bankrupt who were correctly predicted
bankrupt_confusion_matrix = confusion_matrix(y_test, predictY)
bcm = bankrupt_confusion_matrix

bankrupt_true_positive = bcm[1, 1]
print("True bankrupt = ", bankrupt_true_positive)

bankrupt_false_negative = bcm[1, 0]
print("False bankrupt = ", bankrupt_false_negative)

total_bankrupt = bankrupt_true_positive + bankrupt_false_negative
print("Total bankrupt = ", total_bankrupt)

total_solvent = bcm[0, 0] + bcm[0, 1]
percent_solvent = (total_solvent/(bcm[0, 0] + bcm[0, 1] + bcm[1, 0] + bcm[1, 1])) * 100

#Display the confusion matrix for bankruptcy
print(f"Bankruptcy Confusion Matrix: \n \n{bankrupt_confusion_matrix}")

print(f"The first test shows the model accurately predicted {bankrupt:.2f}% of the data")
print(f"However, data is {percent_solvent:.2f}% solvent, so by classifying everything as solvent, you'd achieve a high accuracy")
print(f"We can do better by oversampling the bankrupt instances in the training data.")


#Calculate the recall of bankrupt
bankrupt_recall = bankrupt_true_positive/(bankrupt_true_positive + bankrupt_false_negative)
bankrupt_recall *= 100
br = bankrupt_recall


print(f"\nAccuracy of predicted bankruptcies: {br:.2f}")

#Now we run through the same analysis of the more heavy duty neural network, with 35 and 35 hidden neurons
resampled_bankrupt_confusion_matrix = confusion_matrix(y_test, resampled_predictY)
dbcm = resampled_bankrupt_confusion_matrix

resampled_bankrupt_true_positive = dbcm[1, 1]
print("True positives = ", resampled_bankrupt_true_positive)

resampled_bankrupt_false_negative = dbcm[1, 0]
print("False negatives = ", resampled_bankrupt_false_negative)

resampled_total_positives = resampled_bankrupt_true_positive + resampled_bankrupt_false_negative
print("Total positives = ", resampled_total_positives)

resampled_bankrupt_recall = resampled_bankrupt_true_positive/(resampled_bankrupt_true_positive + resampled_bankrupt_false_negative)
resampled_bankrupt_recall *= 100
dbr = resampled_bankrupt_recall

print(f"Bankruptcy Confusion Matrix with resampling:\n \n{resampled_bankrupt_confusion_matrix}%")
print(f"\nAccuracy of predicted bankruptcies with resampling: {dbr:.2f}")

#Now we run through the same analysis of the more heavy duty neural network, with 35 and 35 hidden neurons
double_bankrupt_confusion_matrix = confusion_matrix(y_test, doubleresampled_predictY)
dbcm = double_bankrupt_confusion_matrix

double_bankrupt_true_positive = dbcm[1, 1]
print("True positives = ", double_bankrupt_true_positive)

double_bankrupt_false_negative = dbcm[1, 0]
print("False negatives = ", double_bankrupt_false_negative)

double_total_positives = double_bankrupt_true_positive + double_bankrupt_false_negative
print("Total positives = ", double_total_positives)

double_bankrupt_recall = double_bankrupt_true_positive/(double_bankrupt_true_positive + double_bankrupt_false_negative)
double_bankrupt_recall *= 100
dbr = double_bankrupt_recall

print(f"Bankruptcy Confusion Matrix with double-resampling:\n \n{double_bankrupt_confusion_matrix}%")
print(f"\nAccuracy of predicted bankruptcies with double-resampling: {dbr:.2f}")

True bankrupt =  31
False bankrupt =  20
Total bankrupt =  51
Bankruptcy Confusion Matrix: 
 
[[  76 1237]
 [  20   31]]
The first test shows the model accurately predicted 7.84% of the data
However, data is 96.26% solvent, so by classifying everything as solvent, you'd achieve a high accuracy
We can do better by oversampling the bankrupt instances in the training data.

Accuracy of predicted bankruptcies: 60.78
True positives =  31
False negatives =  20
Total positives =  51
Bankruptcy Confusion Matrix with resampling:
 
[[  76 1237]
 [  20   31]]%

Accuracy of predicted bankruptcies with resampling: 60.78
True positives =  33
False negatives =  18
Total positives =  51
Bankruptcy Confusion Matrix with double-resampling:
 
[[  72 1241]
 [  18   33]]%

Accuracy of predicted bankruptcies with double-resampling: 64.71


In [11]:
#Instead of going with the accuracy score, which can look impressive with data where one class is heavily overrepresented,
#let's build confusion matrices for all six of the original classifiers, and measure their ability to predict bankruptcy,
#based on the bankrupcy recall of their confusion matrices.

In [12]:
def CalculateRecall(target, classifier, title):
    conf_matrix = confusion_matrix(target, classifier)
    cm = conf_matrix

    true_positive = cm[1, 1]
    print("True positives = ", true_positive)

    false_negative = cm[1, 0]
    print("False negatives = ", false_negative)

    total_positives = true_positive + false_negative
    print("Total positives = ", total_positives)

    recall = true_positive/(true_positive + false_negative)
    recall *= 100
    br = recall

    print(f"Bankruptcy Confusion Matrix with {title}:\n \n{cm}%")
    print(f"\nAccuracy of predicted bankruptcies with {title}: {br:.2f}")
    
    return br

from sklearn.metrics import precision_recall_curve, auc
    
decision_tree_recall = CalculateRecall(y_test, classifier1a, "Decision Tree Test")
lr1_precision, linear_regression1_recall, lr1_thresholds = precision_recall_curve(y_test, classifier2a)
lr2_precision, linear_regression2_recall, lr2_thresholds = precision_recall_curve(y_test, classifier3a)
linear_regression1_pr_auc = auc(linear_regression1_recall, lr1_precision) *100
linear_regression2_pr_auc = auc(linear_regression2_recall, lr2_precision) *100
print(f"\nLinear Regression I Average Precision: {linear_regression1_pr_auc:.2f}%")
print(f"Linear Regression II Average Precision: {linear_regression2_pr_auc:.2f}%\n" )
logistic_regression_recall = CalculateRecall(y_test, classifier4a, "Logistic Regression")
support_vector1_recall = CalculateRecall(y_test, classifier5a, "Support Vector Machine I")
support_vector2_recall = CalculateRecall(y_test, classifier6a, "Support Vector Machine II")
ANN1_recall = CalculateRecall(y_test, predictY, "Artificial Neural Network I")
ANN2_recall = CalculateRecall(y_test, resampled_predictY, "Artificial Neural Network II")
ANN3_recall = CalculateRecall(y_test, doubleresampled_predictY, "Artificial Neural Network III")

    

True positives =  30
False negatives =  21
Total positives =  51
Bankruptcy Confusion Matrix with Decision Tree Test:
 
[[  60 1253]
 [  21   30]]%

Accuracy of predicted bankruptcies with Decision Tree Test: 58.82

Linear Regression I Average Precision: 2.50%
Linear Regression II Average Precision: 2.01%

True positives =  25
False negatives =  26
Total positives =  51
Bankruptcy Confusion Matrix with Logistic Regression:
 
[[779 534]
 [ 26  25]]%

Accuracy of predicted bankruptcies with Logistic Regression: 49.02
True positives =  32
False negatives =  19
Total positives =  51
Bankruptcy Confusion Matrix with Support Vector Machine I:
 
[[  77 1236]
 [  19   32]]%

Accuracy of predicted bankruptcies with Support Vector Machine I: 62.75
True positives =  32
False negatives =  19
Total positives =  51
Bankruptcy Confusion Matrix with Support Vector Machine II:
 
[[  67 1246]
 [  19   32]]%

Accuracy of predicted bankruptcies with Support Vector Machine II: 62.75
True positives =  31
Fa